In [1]:
from load_data_json import LoadDataset 
from preprocess import VectorSpaceModel
import numpy as np
import time
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds, eigsh

## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./data/med/med.json", "./data/med/queries.json", "./data/med/qrels-treceval.txt")

In [3]:
dataset.query_vectors.shape

(13004, 30)

# Vector Space Model ranked retrieval

Create the `vsm()` object.

In [4]:
vsm = VectorSpaceModel(dataset.doc_matrix)

In [5]:
vsm.A.shape

(13004, 1033)

In [6]:
x = vsm.A.T @ dataset.query_vectors[:,0] 

In [7]:
s = np.zeros(13004)

In [8]:
dataset.query_vectors[:,0].shape

(13004, 1)

In [16]:
x = vsm.A.T @ dataset.query_vectors[:,0]

<1033x1 sparse matrix of type '<class 'numpy.float64'>'
	with 71 stored elements in Compressed Sparse Row format>

In [10]:
vsm.CompareToAb(dataset.query_vectors[:,0], k=300)

[-0.01092089 -0.00671323  0.00079087 ... -0.00529139  0.00839204
 -0.00165579]


In [11]:
u, sigma, vt = svds(vsm.A)

In [12]:

u = u[:, ::-1]
sigma = sigma[::-1]
vt = vt[:, ::-1]
scores = (u.T @ dataset.query_vectors[:,0])[:,0]
scores

array([ 0.03268063,  0.0265402 ,  0.03513878, -0.01427257,  0.03915239,
       -0.04770335])

In [13]:
scores = scores * sigma

In [14]:
vt.T @ scores

array([-0.00708434, -0.00332165,  0.00148535, ..., -0.00512498,
       -0.00283414, -0.00244149])